In [6]:
from mimas.alphapept.find_features import find_features
from mimas.alphapept.load_mzml_data import load_mzml_data
from mimas.alphapept.find_features import find_features
import pandas as pd
import os
from toolsets.search import quick_search_sorted

def map_features(feature_mix, std_list_mix, adducts):
    feature_mix.sort_values(by = 'mz', inplace=True, ascending=True)
    mix_matched = pd.DataFrame()
    for index, row in std_list_mix.iterrows():
        compound_matched = pd.DataFrame()

        for a in adducts:
            adduct_matched = quick_search_sorted(feature_mix, 'mz', row[a]-0.005, row[a]+0.005)
            if len(adduct_matched)>0:
                # adduct_matched.insert(0, 'reference_mz', row[a])
                adduct_matched.insert(1, 'reference_name', row['Name'])
                adduct_matched.insert(1, 'reference_mz', row[a])
                adduct_matched.insert(1,'reference_inchikey', row['inchikey'])
                adduct_matched.insert(2, 'reference_adduct', a)

                # adduct_matched.insert(3, 'reference_rt', row['reference_rt'])
                adduct_matched.insert(4, 'reference_smiles', row['smiles'])
                adduct_matched.insert(6, 'reference_mix', row['Mix'])
                adduct_matched.insert(7, 'reference_rt', row['RT'])
                compound_matched  = pd.concat([compound_matched, adduct_matched], ignore_index=True)
            # return compound_matched
        if row['RT']==row['RT'] and len(compound_matched)>0:
            rt_offsets = abs(compound_matched['rt_apex']-row['RT'])
            compound_matched['rt_offset']=rt_offsets
            compound_matched = compound_matched[compound_matched['rt_offset']<5/60]
            compound_matched.drop(columns=['rt_offset'], inplace =True)
        elif len(compound_matched)>0:
            compound_matched.sort_values(by = 'ms1_int_max_apex', inplace=True, ascending=False)
            compound_matched.drop_duplicates(subset=['reference_adduct'], keep = 'first', inplace=True)

            # compound_matched = pd.concat([compound_matched, adduct_matched], ignore_index=True)
        mix_matched = pd.concat([mix_matched, compound_matched], ignore_index=True)
    return(mix_matched)

# extract features

In [4]:
file_name = "/Users/fanzhoukong/Downloads/Winston0014_MX753152_posBA_Nala-Wallace-Week-6-078.mzml"
ms_file_data = load_mzml_data(file_name)
feature_temp =find_features(ms_file_data)
#use for loop to extract all features

/Users/fanzhoukong/Documents/GitHub/lead_finder/mimas/alphapept/find_features.py:284: RuntimeWarning: divide by zero encountered in divide
  feature_sn = feature_apex_xic / feature_noise * 2
/Users/fanzhoukong/Documents/GitHub/lead_finder/mimas/alphapept/find_features.py:284: RuntimeWarning: invalid value encountered in divide
  feature_sn = feature_apex_xic / feature_noise * 2


# map features

In [ ]:
std_list = pd.read_csv(os.path.join('/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT', 'enlongated_list.csv'))
mapped_all = pd.DataFrame()
for mix in std_list['Mix'].unique():
    feature_mix = pd.read_csv('path_to_features')
    std_list_mix = std_list[std_list['Mix']==mix]
    mapped_features = map_features(feature_mix, std_list_mix, adducts=['[M+H]+'])# extedn adducts to all adducts you want
    mapped_all = pd.concat([mapped_all, mapped_features], ignore_index=True)